In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
PATH_PROJECT_DATA = Path('../data')
GRAND_SLAMS = ['wimbledon', 'roland-garros', 'us-open', 'australian-open']

# Leemos los partidos

In [ ]:
df_matches = pd.read_csv(PATH_PROJECT_DATA / 'all_matches.csv')

# Filtramos solo los partidos de Grand Slam de individuales posteriores al año 1990 y eliminamos columnas que no nos den información

In [ ]:
columns_to_drop = ['end_date', 'location', 'prize_money',
                   'currency', 'num_sets', 'games_against',
                   'doubles', 'masters', 'nation']
                   
is_grand_slam = df_matches['tournament'].isin(GRAND_SLAMS)
is_individual = ~df_matches['player_id'].str.contains('_')
is_after_1990 = df_matches['year'] >= 1990

df_matches_grand_slam = (
    df_matches[is_grand_slam & is_individual & is_after_1990]
    .drop(columns=columns_to_drop)
    .copy()
)

# Rellenamos los `seed` que están a nulo con el valor 1000

Esto es porque los NA corresponde a gente que no está en el top, por tanto de cara a un modelo es interesante ponerles un valor alto para que sepa que están lejos del número 1.

También nos interesa dejarlo como columna numérica de cara a un modelo, eliminando los valores Q, WC, LL...

In [ ]:
SEEDS_TO_DROP = ['Q', 'WC', 'PR', 'LL', 'AL', '1 WC']
SEEDS_FILLNA = '1000'

df_matches_grand_slam['seed'] = (
    df_matches_grand_slam['seed'].fillna(SEEDS_FILLNA)
)

df_matches_grand_slam.loc[df_matches_grand_slam['seed']
                          .isin(SEEDS_TO_DROP), 'seed'] = SEEDS_FILLNA

df_matches_grand_slam['seed'] = df_matches_grand_slam['seed'].astype(int)

# Eliminamos los partidos de los cuales no hay información más allá del resultado

In [ ]:
df_matches_grand_slam_dropna = df_matches_grand_slam.dropna(how='any').copy()

# Cruzamos los players para tener información de los dos jugadores en la misma fila

- Creamos la columna `players` para poder tener a cada pareja identificada alfabeticamente y en una sola columna
- Creamos dos dataframes, `df_player` y `df_opponent` donde vamos a tener por separado la información del jugador 1 y del 2
- Cruzamos los dos para tener toda la información en un solo dataframe y eliminamos las columnas que tengan información duplicada o que no nos interesen para dejarlo todo limpio

In [ ]:
df_matches_grand_slam_dropna['players'] = (
    df_matches_grand_slam_dropna[['player_id', 'opponent_id']]
    .apply(lambda x: ' '.join(sorted([x['player_id'], x['opponent_id']])),
           axis=1)
)

df_player = (df_matches_grand_slam_dropna
             .groupby(['players', 'year', 'tournament'])
             .head(1)
             .rename(columns=dict(player_name='player_1',
                                  player_victory='player_victory_1',
                                  won_first_set='won_first_set_1'))
             .drop(columns=['player_id', 'opponent_id', 'opponent_name']))

df_opponent = (df_matches_grand_slam_dropna
               .groupby(['players', 'year', 'tournament'])
               .tail(1)
               .rename(columns=dict(player_name='player_2'))
               .drop(columns=['player_id', 'opponent_id',
                              'tiebreaks_total', 'total_points',
                              'round', 'duration',
                              'opponent_name', 'court_surface',
                              'won_first_set']))

df_matches_grand_slam_cleaned = (
    df_player
    .merge(df_opponent.drop(columns=['player_victory',
                                     'start_date',
                                     'round_num']),
           on=['players', 'year', 'tournament'],
           how='inner',
           suffixes=('_1', '_2'))
)

# Dataframe de jugadores limpio

In [ ]:
df_players_cleaned = pd.concat(
    [df_player
     .rename(columns=dict(player_1='player_name',
                          player_victory_1='player_victory'))
     .drop(columns=['tiebreaks_total', 'total_points',
                    'round', 'duration',
                    'court_surface', 'won_first_set_1',
                    'players']),
     df_opponent
     .rename(columns=dict(player_2='player_name'))
     .drop(columns=['players'])]
)

# Guardamos los datos limpios

In [ ]:
df_matches_grand_slam_cleaned.to_csv(PATH_PROJECT_DATA
                                     / 'matches_grand_slam_cleaned.csv',
                                     index=None)

df_players_cleaned.to_csv(PATH_PROJECT_DATA
                          / 'players_cleaned.csv',
                          index=None)